In [2]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/cambrian_reco/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
import numpy as np
import pandas as pd
import json
import requests
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/cambrian_reco/')

In [3]:
from sentence_transformers import SentenceTransformer

In [4]:
baai = SentenceTransformer('BAAI/bge-large-en-v1.5')

In [202]:
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

In [203]:
with PostgresDatabaseOperation() as cursor:
    sql = 'SELECT topic_id, description, approach_name FROM topic_internal_representations'
    cursor.execute(sql)
    results = cursor.fetchall()

In [204]:
rep = pd.DataFrame(results, columns = ['topic_id', 'description', 'approach_name'])

In [205]:
descriptions = list(rep.description.values)

In [23]:
with PostgresDatabaseOperation() as cursor:
    candidate_embeddings = 'SELECT ca.article_id, embedding FROM candidate_articles ca LEFT JOIN embeddings e ON ca.article_id = e.article_id'
    cursor.execute(candidate_embeddings)
    results = cursor.fetchall()
candidate_embeddings_df = pd.DataFrame(results, columns = ['article_id', 'embedding'])

In [38]:
candidate_embeddings_df['embedding'] = candidate_embeddings_df['embedding'].apply(lambda x: json.loads(x))

candidate_embeddings_df['embedding'] = candidate_embeddings_df['embedding'].apply(lambda x: np.array([x]))

In [206]:
len(descriptions)

180

In [207]:
len(set(descriptions))

180

In [208]:
embs = {}
for cur_des in descriptions:
    embs[cur_des] = baai.encode(cur_des)

In [22]:
from sklearn.metrics.pairwise import cosine_similarity

### Finding distribution

In [209]:
rep[(rep.topic_id == 3) & (rep.approach_name == 'gpt4-keywords-v2')].description.values[0]

'Financial Independence, Retire Early, FIRE movement, passive income, personal finance, early retirement, savings rate, investment strategy, financial planning, compounding, frugality, 4% rule, equity investment, mutual funds, financial literacy, 25x rule, retirement planning, stock market, index funds, financial freedom, diversified portfolio, ETFs (Exchange Traded Funds), SWR (Safe Withdrawal Rate), wealth creation, net worth, \n\nRetirement corpus, Financial independence community, debt ratio, asset allocation, financial coaching, HDFC (Housing Development Finance Corporation), SIP (Systematic Investment Plan), SBI (State Bank of India), PF (Provident Fund), PPF (Public Provident Fund), INR (Indian Rupee), LIC (Life Insurance Corporation), RBI (Reserve Bank of India), NPS (National Pension System), retirement age, FIRE calculator, loan EMIs, pension funds, UTI Mutual Fund, expense ratio, budgeting, early retirees, Aditya Birla Sun Life AMC,  HDFC Bank, debt-to-income ratio, income s

In [130]:
rep[(rep.topic_id == 3) & (rep.approach_name == 'gpt3-article')].description.values[0]

'. Introduction: Retirement Planning is a crucial aspect of Personal Finance that focuses on allocating savings and investments to ensure financial security during retirement years. This category covers various aspects such as saving strategies investment options tax planning pension schemes and other relevant topics.  . Technical Aspects and Background: To understand Retirement Planning effectively it is important to have a good grasp of financial concepts like compound interest inflation risk tolerance asset allocation and long-term investment strategies. Familiarity with retirement accounts (e.g. (k) Individual Retirement Accounts (IRAs) and Pension Plans) is also essential.  . Common Examples: Retirement Planning encompasses a wide range of topics including: - Determining the target retirement corpus - Deciding the appropriate retirement age - Analyzing different investment options such as stocks bonds mutual funds and real estate - Understanding social security benefits and govern

In [135]:
from tqdm.notebook import tqdm

In [137]:
topic_closest_articles = {}
for topic_id in tqdm(range(1,46)):
    dfs = []
    for approach_name in ['basic', 'gpt3-article', 'gpt4-keywords']:
        topic_name = rep[(rep.topic_id == topic_id) & (rep.approach_name == approach_name)].description.values[0]
        topic_embedding = embs[topic_name]
        candidate_embeddings_df['dist'] = candidate_embeddings_df['embedding'].apply(lambda x: cosine_similarity([topic_embedding], x)[0][0])
        cur_df = candidate_embeddings_df.nlargest(200, 'dist')
        cur_df['approach_name'] = approach_name
        cur_df['topic_id'] = topic_id
        dfs.append(cur_df)
    topic_closest_articles[topic_id] = pd.concat(dfs)

  0%|          | 0/45 [00:00<?, ?it/s]

In [138]:
closest_df = pd.concat(list(topic_closest_articles.values()))

In [142]:
closest_df.groupby('approach_name')['dist'].describe()

,count,mean,std,min,25%,50%,75%,max
approach_name,,,,,,,,
basic,9000.0,0.616013,0.038191,0.516383,0.589847,0.611241,0.640277,0.802251
gpt3-article,9000.0,0.647444,0.039323,0.572647,0.618751,0.642263,0.669851,0.849650
gpt4-keywords,9000.0,0.630213,0.031037,0.567650,0.608285,0.626185,0.648683,0.800489


In [148]:
closest_df[(closest_df.topic_id == 7) & (closest_df.approach_name == 'gpt3-article')]

,article_id,embedding,dist,approach_name,topic_id
1003,65316f161e5cc42b1b13eed5,"[[-0.002882043, 0.060545474, 0.028071329, -0.0...",0.792237,gpt3-article,7
7783,6555cbea4b13023f934a2e31,"[[-0.5028681, 0.78035575, 0.078065105, -0.0328...",0.788759,gpt3-article,7
5565,652ebbdf1e5cc42b1b139bb2,"[[0.011557011, 0.023140926, 0.0049594445, -0.0...",0.782999,gpt3-article,7
6270,6555cb5e4b13023f9349cd77,"[[-0.29583028, 0.27324086, 0.271881, 0.4514497...",0.773002,gpt3-article,7
7964,6555cc2c4b13023f934a5daf,"[[0.94163704, 0.92136866, -0.33977902, -0.1181...",0.764011,gpt3-article,7
...,...,...,...,...,...
4074,6555ca684b13023f934937f8,"[[0.565909, -0.063396, -0.017944565, 0.4060316...",0.666279,gpt3-article,7
11525,657fee108dc7b10db3e78f84,"[[-0.010704231, 0.029255545, -8.001487e-05, 0....",0.666224,gpt3-article,7
2592,6555c7f24b13023f9348f155,"[[-0.0051064994, 0.37201166, 0.36548826, 0.096...",0.666062,gpt3-article,7
9657,6571c34b25cc9a343f03f5f6,"[[-0.012811391, 0.040823206, -0.014343811, -0....",0.665796,gpt3-article,7


In [195]:
custom_keyword = '''
BSE (Bombay Stock Exchange), NSE (National Stock Exchange), SEBI (Securities and Exchange Board of India), RIL (Reliance Industries Limited), TCS (Tata Consultancy Services), Infosys, HDFC Bank, ICICI Bank, IPO (Initial Public Offering), Mutual Funds, Bonds, Equity, Dividends, Blue-chip stocks, Mid-cap stocks, Small-cap stocks, Share Price, Market Capitalisation, NIFTY, SENSEX, Intraday Trading, Profit Margin, Stockbroker, Open Interest, Portfolio, Derivatives, Bull Market, Bear Market, Volatility, Market Trend, Earnings Per Share, FII (Foreign Institutional Investors), DII (Domestic Institutional Investors), RBI (Reserve Bank of India), FMCG (Fast Moving Consumer Goods), IT Sector, Banking Sector, Insurance Sector, Real Estate Sector, Automobile Sector, Pfizer, Market Predictions, Analyst Ratings, Market Close, Market Open, Financial Statements, Quarterly Results, Fiscal Year, Dalal Street, Market Correction.
'''

In [196]:
custom_emb = baai.encode(custom_keyword)

In [197]:
custom_emb

array([-0.03307178,  0.011424  ,  0.02156772, ...,  0.00870629,
        0.00116783, -0.03813513], dtype=float32)

In [183]:
from copy import deepcopy

In [184]:
custom_df = deepcopy(candidate_embeddings_df)

In [198]:
custom_df['dist'] = custom_df['embedding'].apply(lambda x: cosine_similarity([custom_emb], x)[0][0])

In [189]:
rep[rep.approach_name == 'basic']

,topic_id,description,approach_name
9,1,Personal Finance: Budgeting & Saving,basic
10,2,Personal Finance: Debt Management,basic
11,3,Personal Finance: Retirement Planning,basic
12,4,Personal Finance: Tax Planning,basic
13,5,"Personal Finance: Insurance (Life, Health, Aut...",basic
14,6,Personal Finance: Real Estate & Home Ownership,basic
15,7,Investment: Stock Market,basic
16,8,Investment: Equities,basic
17,9,Investment: Mutual Funds,basic
18,10,Investment: ETFs,basic


In [191]:
closest_df[closest_df.topic_id == 7].groupby('approach_name')['dist'].mean()

approach_name
basic            0.660596
gpt3-article     0.690469
gpt4-keywords    0.671437
Name: dist, dtype: float64

In [194]:
closest_df[(closest_df.topic_id == 7) & (closest_df.approach_name == 'gpt3-article')]

,article_id,embedding,dist,approach_name,topic_id
1003,65316f161e5cc42b1b13eed5,"[[-0.002882043, 0.060545474, 0.028071329, -0.0...",0.792237,gpt3-article,7
7783,6555cbea4b13023f934a2e31,"[[-0.5028681, 0.78035575, 0.078065105, -0.0328...",0.788759,gpt3-article,7
5565,652ebbdf1e5cc42b1b139bb2,"[[0.011557011, 0.023140926, 0.0049594445, -0.0...",0.782999,gpt3-article,7
6270,6555cb5e4b13023f9349cd77,"[[-0.29583028, 0.27324086, 0.271881, 0.4514497...",0.773002,gpt3-article,7
7964,6555cc2c4b13023f934a5daf,"[[0.94163704, 0.92136866, -0.33977902, -0.1181...",0.764011,gpt3-article,7
...,...,...,...,...,...
4074,6555ca684b13023f934937f8,"[[0.565909, -0.063396, -0.017944565, 0.4060316...",0.666279,gpt3-article,7
11525,657fee108dc7b10db3e78f84,"[[-0.010704231, 0.029255545, -8.001487e-05, 0....",0.666224,gpt3-article,7
2592,6555c7f24b13023f9348f155,"[[-0.0051064994, 0.37201166, 0.36548826, 0.096...",0.666062,gpt3-article,7
9657,6571c34b25cc9a343f03f5f6,"[[-0.012811391, 0.040823206, -0.014343811, -0....",0.665796,gpt3-article,7


In [200]:
custom_df.nlargest(100, 'dist')

,article_id,embedding,dist
11377,657fb5d58f821120f67a0986,"[[-0.017665314, 0.022280756, 0.0022594065, -0....",0.731016
2920,6555c8074b13023f9348fd11,"[[-0.27023032, 0.8819144, 0.22423579, -0.02480...",0.724294
8097,6571c25625cc9a343f0380d3,"[[-0.011433897, 0.012732562, 0.016246602, 0.00...",0.719137
7880,6571c2b725cc9a343f03aac5,"[[-0.00085473905, 0.03772602, 0.0032542017, -0...",0.719054
9400,6566e1feb878114e6959770f,"[[0.011448824, 0.04779625, 0.013412369, 0.0207...",0.707143
...,...,...,...
9062,6560d2a1e1d8342798a467da,"[[0.0052035474, 0.018957783, 0.007575832, 0.02...",0.669719
9927,6572f743cbb3242a0060e81b,"[[0.015935594, 0.013224952, -0.006161751, -0.0...",0.669666
6816,6570402153e8a6117bd3090a,"[[-0.012105277, 0.03282544, -0.035824846, -0.0...",0.669247
5565,652ebbdf1e5cc42b1b139bb2,"[[0.011557011, 0.023140926, 0.0049594445, -0.0...",0.669177


### Saving embeddings to db

In [210]:
representation_embeddings = {}

In [212]:
for row in rep.itertuples():
    representation_embeddings[row.description] = {}
    representation_embeddings[row.description]['topic_id'] = row.topic_id
    representation_embeddings[row.description]['approach_name'] = row.approach_name
    representation_embeddings[row.description]['embedding'] = [float(x) for x in list(embs[row.description])]

In [222]:
rep[(rep.approach_name == 'gpt4-keywords-v2') & (rep.topic_id == 7)].description.values[0]

'BSE (Bombay Stock Exchange), NSE (National Stock Exchange), Sensex, Nifty, IPO (Initial Public Offering), Bear Market, Bull Market, Stock Broker, Portfolio, Equities, Blue Chip Stocks, Market Capitalization, Dividend, Futures Contract, Options Contract, Day Trading, Commodity Market, SEBI (Securities and Exchange Board of India), Stock Split, Intraday Trading, Technical Analysis, Derivatives, Mutual Funds, Earnings Report, FDI (Foreign Direct Investment), Reliance Industries, Infosys, Tata Consultancy Services, L&T Finance, ICICI Bank, HDFC Bank, Yes Bank, PSU (Public Sector Unit) Banks, Cipla, Wipro, Tech Mahindra, Automobile Stocks, Pharma Stocks, Sensex Sensitive stocks, Market Correction, FMCG (Fast-Moving Consumer Goods) Companies, Life Insurance Corporation, Real Estate Stocks, ITC, Equity Shares, Financial Year Earnings, Trading Volume, RBI (Reserve Bank of India).'

In [217]:
rep.head(40)

,topic_id,description,approach_name
0,1,Personal Finance: Budgeting & Saving,basic
1,2,Personal Finance: Debt Management,basic
2,3,Personal Finance: Retirement Planning,basic
3,4,Personal Finance: Tax Planning,basic
4,44,"Renewable Portfolio Standards, Sustainable Ass...",gpt4-keywords
5,1,"Smart Finances, Asset Management, Capital Grow...",gpt4-keywords
6,2,"Lending Rates, Financial Risk, Asset Liquidati...",gpt4-keywords
7,3,"Retirement Security, IRDAI Pension Policies, E...",gpt4-keywords
8,4,"Wealth management tax strategies, Inheritance ...",gpt4-keywords
9,5,"Risk cover, Third party liability, Accidental ...",gpt4-keywords


In [215]:
with PostgresDatabaseOperation() as cursor:
    sql = '''UPDATE topic_internal_representations SET description_embedding = %s, embedding_model_name = 'BAAI/bge-large-en-v1.5'
    WHERE topic_id = %s AND approach_name = %s'''
    for description, val in representation_embeddings.items():
        cursor.execute(sql, (val['embedding'], val['topic_id'], val['approach_name']))

In [153]:
representation_embeddings['Personal Finance: Budgeting & Saving']

{'topic_id': 1,
 'approach_name': 'basic',
 'embedding': array([ 0.01654258,  0.00749471, -0.01307069, ...,  0.00264251,
        -0.00887567,  0.00594615], dtype=float32)}

In [129]:
closest_df.groupby('topic_id')['dist'].describe()['mean'].mean()

0.6160128750801284

In [134]:
closest_df.groupby('topic_id')['dist'].describe()['mean'].mean()

0.6160128750801284

In [123]:
closest_df.groupby('topic_id')['dist'].describe()['mean'].mean()

0.6302131230704672

In [131]:
closest_df.article_id.nunique()

3582

In [132]:
closest_df.groupby('topic_id')['dist'].min()

topic_id
45    0.516383
Name: dist, dtype: float64

In [161]:
from datetime import datetime

In [173]:
datetime.now().minute % 3

1

### Reco testing

In [174]:
from src.ArticleSelectionService import ArticleSelectionService

In [226]:
rep[(rep.approach_name == 'gpt4-keywords-v2') & (rep.topic_id.isin([1,3,5]))].description.values

array(['Policy, Premium, Coverage, Claim, Deductible, Liability, Insurance Regulatory and Development Authority (IRDA), Life Insurance Corporation (LIC), Max Life Insurance, Policyholder, Insurer, Bajaj Allianz, Auto Insurance, Health Insurance, Life Insurance, Accident Cover, Whole Life Policy, Term Insurance, Underwriting, General Insurance, Oriental Insurance, United India Insurance, New India Assurance, Exclusions, Prudential Life Insurance, Risk, Payout, AEGON Life, Apollo Munich, Bharti AXA, Claim Settlement Ratio, Critical Illness Cover, Apollo Munich Health Insurance, Cashless Hospitalisation, ICICI Prudential Life Insurance, Group Health Insurance, HDFC Life, Kotak Mahindra Life Insurance, TATA AIG, Reliance Life Insurance, SBI Life, Star Health and Allied Insurance, Travel Insurance, Universal Sompo, Third-Party Insurance, Vehicle Insurance, Religare Health Insurance, Co-Payment, National Insurance Company Limited (NICL).',
       'Financial Independence, Retire Early, FIRE m

In [228]:
2 in [2, 3]

True

In [223]:
rep[rep.approach_name == 'basic']

,topic_id,description,approach_name
0,1,Personal Finance: Budgeting & Saving,basic
1,2,Personal Finance: Debt Management,basic
2,3,Personal Finance: Retirement Planning,basic
3,4,Personal Finance: Tax Planning,basic
14,7,Investment: Stock Market,basic
15,8,Investment: Equities,basic
16,9,Investment: Mutual Funds,basic
17,11,Investment: Real Estate Investment,basic
18,12,Investment: Bonds & Fixed Income,basic
19,13,"Investment: Commodities (Gold, Silver, Oil, etc.)",basic


In [179]:
rep[rep.topic_id.isin([9, 7, 12, 34, 16])]

,topic_id,description,approach_name
6,7,"BSE Sensex trends, Nifty fluctuations, Blue ch...",gpt4-keywords
8,9,"Equity mutual funds, Debt mutual funds, Invest...",gpt4-keywords
15,7,Investment: Stock Market,basic
17,9,Investment: Mutual Funds,basic
20,12,Investment: Bonds & Fixed Income,basic
25,12,"Federal Notes, Bond Liquidity, Bonds Interest,...",gpt4-keywords
29,16,"Savings Plan, Retail Banking, Bank Interest, A...",gpt4-keywords
49,34,"Digital transactions, Innovative Fintech, Cryp...",gpt4-keywords
51,7,. Introduction: Stock Market is a category wit...,gpt3-article
64,16,. Introduction: Savings and current accounts a...,gpt3-article


In [110]:
rep[rep.topic_id == 23]

,topic_id,description,approach_name
36,23,"Indian M&A Market, Corporate Takeovers, Busine...",gpt4-keywords
90,23,. Introduction: Mergers Acquisitions (MA) is ...,gpt3-article
112,23,Business Finance: Mergers & Acquisitions,basic


In [24]:
candidate_embeddings_df.head(3)

,article_id,embedding
0,652fcebd1e5cc42b1b13a329,"[-0.017601214,0.047247745,0.0034381868,-0.0040..."
1,652fcec71e5cc42b1b13a32e,"[-0.017607532,0.062459026,-0.003675223,0.02359..."
2,652fcecd1e5cc42b1b13a331,"[-0.018655157,0.06946019,-0.036561742,-0.00571..."


In [101]:
wealth_embeddings = embs[retirement_article]

In [82]:
wealth_embeddings

array([ 0.00191034,  0.03004712, -0.03119135, ...,  0.00131262,
        0.04644846, -0.00237857], dtype=float32)

In [102]:
candidate_embeddings_df['dist'] = candidate_embeddings_df['embedding'].apply(lambda x: cosine_similarity([wealth_embeddings], x)[0][0])

In [105]:
candidate_embeddings_df.nlargest(40, 'dist')

,article_id,embedding,dist
4919,6555cace4b13023f93496e14,"[[0.5749195, 1.0778024, -0.26991743, 0.1276341...",0.769686
1328,653680301e5cc42b1b14424f,"[[-0.009467053, 0.02348751, -0.04035513, -0.01...",0.768438
2061,6555c7bf4b13023f9348dfae,"[[0.34918803, 0.57320166, -0.730244, 0.3927492...",0.764068
4372,6555ca8d4b13023f934945d0,"[[0.55477285, 0.44564235, -0.69903535, 0.32921...",0.761970
8993,655d6ed8e1d8342798a448dd,"[[0.04076252, 0.01592794, -0.0531718, -0.00088...",0.749547
6179,6555cb4b4b13023f9349c130,"[[-0.31791657, -0.11994168, -0.6197266, 0.1515...",0.741495
8393,6555cc6b4b13023f934a8c3d,"[[0.46907672, 0.049958475, -0.64199656, 0.7078...",0.739334
7374,6555cbb44b13023f934a08e5,"[[0.55364025, 0.8662387, -0.62555796, 0.452198...",0.737686
2949,6555c80c4b13023f9349007e,"[[0.0076786536, 0.5442379, -0.785812, 0.274953...",0.736820
4740,6555cab84b13023f93495efe,"[[0.2782915, 0.71977705, -0.29234326, -0.15811...",0.735918


In [ ]:
embs[descriptions]

In [21]:
embs[descriptions[0]]

array([-0.02382286,  0.02176197,  0.01861183, ..., -0.00623289,
       -0.00191846,  0.01633771], dtype=float32)